In [87]:
from __future__ import print_function
import sys
import os
import logging
import six
from radiomics import featureextractor, getFeatureClasses
import radiomics
%config Completer.use_jedi = False
import SimpleITK as sitk
import nibabel as nib
import glob
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
import numpy as np
import logging

In [88]:
ls_image =sorted(glob.glob('./*_NAT*/*.nii'))

# Global: 3D

In [89]:
# Alternative: use hardcoded settings (separate for settings, input image types and enabled features)
settings = {}
settings['binWidth'] = 25
#settings['resampledPixelSpacing'] = [2,2]  # en 2D, on travaille dans un plan
settings['resampledPixelSpacing'] = [2, 2, 2]  # This is an example for defining resampling (voxels with size 3x3x3mm)
settings['interpolator'] = 'sitkBSpline'
settings['verbose'] = True
settings['force2D'] = True
settings['force2Ddimension'] = 0


extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': False,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': [2, 2, 2],
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': True,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': True,
 'binWidth': 25,
 'verbose': True}

In [80]:
all_dat = pd.DataFrame()
extractor.disableAllFeatures
li_featureClass = ['firstorder', 'shape','glrlm', 'glszm', 'gldm', 'ngtdm']
for featureClass in li_featureClass:
    extractor.enableFeatureClassByName(featureClass)
#On ne pourrait pas avoir les 2D...
extractor.enableFeaturesByName(**{"glcm" : ['Autocorrelation',
    'ClusterProminence',
    'ClusterShade',
    'ClusterTendency',
    'Contrast',
    'Correlation',
    'DifferenceAverage',
    'DifferenceEntropy',
    'DifferenceVariance',
    'Id',
    'Idm',
    'Idmn',
    'Idn',
    'Imc1',
    'Imc2',
    'InverseVariance',
    'JointEnergy',
    'JointEntropy',
    'MCC',
    'MaximumProbability',
    'SumAverage',
    'SumEntropy',
    'SumSquares'
]})

logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR) #Dégager les message inutiles (car on y a fait attention)

#OUILLE LES PATIENTS S'APPELLENT ./DATASET et pas en slices???

for imageName in tqdm.tqdm(ls_image):
    maskName = imageName.replace('_NAT','').replace('.nii','_masked.nii')
    classe_name = imageName.split('/')[-2].split('_')[1]
    temps_inj = imageName.split('_')[-1].split('.')[0]

    patient_num = imageName.split('/')[-1].split('_')[0]
    #print(patient_num) 

        
    mask_sitk = sitk.ReadImage(maskName) !=  0  ### ATTENTION C'est important ici on s'interesse au masque et non a l'image masquée
    # print(imageName)
    # print(mask_sitk.GetSize())
    #print(maskName)
    mr_sitk = sitk.ReadImage(imageName)
    #print("nbe voxels :")
    #print(sitk.GetArrayFromImage(mask_sitk).sum())


    featureVector = extractor.execute(mr_sitk, mask_sitk)
    df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
    df_idx['classe_name'] = classe_name
    df_idx['temps_inj'] = temps_inj
    df_idx['patient_num'] = patient_num
    df_idx = pd.DataFrame(df_idx).T
    all_dat = pd.concat([df_idx,all_dat])
    


100%|██████████| 550/550 [06:08<00:00,  1.49it/s]


In [81]:
all_dat.to_csv('./liver_tumors_all_LLB.csv')
all_dat.to_excel('./global_excel.xlsx', index = False)


# Multislice

In [60]:
settings = {}
settings['binWidth'] = 25
settings['resampledPixelSpacing'] = [2,2]
#settings['resampledPixelSpacing'] = [3, 3, 3]  # This is an example for defining resampling (voxels with size 3x3x3mm)
settings['interpolator'] = 'sitkBSpline'
settings['verbose'] = True
settings['force2D'] = True
settings['force2Ddimension'] = 0   # ne pas oublier de le changer : fait: ce n'est plus True


extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': False,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': [2, 2],
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': True,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': True,
 'binWidth': 25,
 'verbose': True}

In [61]:
al_dat_slice = pd.DataFrame()
label=extractor.settings.get('label')
label=1
minDims=2
extractor.disableAllFeatures()
li_featureClass = ['firstorder','shape2D', 'glrlm', 'glszm', 'gldm', 'ngtdm']
for featureClass in li_featureClass:
    extractor.enableFeatureClassByName(featureClass)
extractor.enableFeaturesByName(**{"glcm": ['Autocorrelation',
    'ClusterProminence',
    'ClusterShade',
    'ClusterTendency',
    'Contrast',
    'Correlation',
    'DifferenceAverage',
    'DifferenceEntropy',
    'DifferenceVariance',
    'Id',
    'Idm',
    'Idmn',
    'Idn',
    'Imc1',
    'Imc2',
    'InverseVariance',
    'JointEnergy',
    'JointEntropy',
    'MCC',
    'MaximumProbability',
    'SumAverage',
    'SumEntropy',
    'SumSquares'
]})
logger = logging.getLogger("radiomics.glcm")
logger.setLevel(logging.ERROR) #Dégager les message inutiles (car on y a fait attention)

for imageName in tqdm.tqdm(ls_image):
    
        #print(imageName)
        maskName = imageName.replace('_NAT','').replace('.nii','_masked.nii')
        
        
        classe_name = imageName.split('/')[-2].split('_')[1]
        temps_inj = imageName.split('_')[-1].split('.')[0]
        patient_num = imageName.split('/')[-1].split('_')[0]
    
        
        mask_sitk = sitk.ReadImage(maskName) !=  0  ### ATTENTION C'est important ici on s'interesse au masque et non a l'image masquée
        mr_sitk = sitk.ReadImage(imageName)
        
        

        for slice_num in range(mask_sitk.GetSize()[-1]):
            
            
            slice_mask = mask_sitk[:,:,slice_num]
            slice_img = mr_sitk[:,:,slice_num]
            
            lsif = sitk.LabelStatisticsImageFilter()
            lsif.Execute(slice_img, slice_mask)
            boundingBox = np.array(lsif.GetBoundingBox(label))
            ndims = np.sum((boundingBox[1::2] - boundingBox[0::2] + 1) > 3)  # UBound - LBound + 1 = Size
            #print('ndims ',ndims)
            if (sitk.GetArrayFromImage(slice_mask).sum() > 0) & (ndims >= minDims):
               # slice_old = slice_img
                slice_img = mr_sitk[:,:,slice_num]
                
                
                
                featureVector = extractor.execute(slice_img, slice_mask)
                featureVector['slice_num'] = slice_num
                df_idx = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in featureVector.items() ])).iloc[0]
                df_idx['classe_name'] = classe_name
                df_idx['temps_inj'] = temps_inj
                df_idx['patient_num'] = patient_num
               # al_dat_slice = al_dat_slice.append(df_idx)
                df_idx = pd.DataFrame(df_idx).T
                al_dat_slice = pd.concat([df_idx,al_dat_slice])
     

100%|██████████| 550/550 [10:36<00:00,  1.16s/it]


In [37]:
print(extractor.enabledFeatures)

{'firstorder': [], 'shape2D': [], 'glrlm': [], 'glszm': [], 'gldm': [], 'ngtdm': [], 'glcm': ['Autocorrelation', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'Id', 'Idm', 'Idmn', 'Idn', 'Imc1', 'Imc2', 'InverseVariance', 'JointEnergy', 'JointEntropy', 'MCC', 'MaximumProbability', 'SumAverage', 'SumEntropy', 'SumSquares']}


In [284]:
a=sitk.GetArrayFromImage(slice_mask)
print(a.sum(axis=1))
print(slice_old.GetDimension())
print(extractor.settings)
label=extractor.settings.get('label')
label=1
lsif = sitk.LabelStatisticsImageFilter()
lsif.Execute(slice_img, slice_mask)
boundingBox = np.array(lsif.GetBoundingBox(label))
print(boundingBox)
ndims = np.sum((boundingBox[1::2] - boundingBox[0::2] + 1) > 3)  # UBound - LBound + 1 = Size
print('ndims ',ndims)
a = boundingBox[1::2] - boundingBox[0::2]
print(a)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 3 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2
{'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': [2, 2], 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': True, 'force2Ddimension': True, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'verbose': True}
[225 227 127 129]
ndims  0
[2 2]


In [289]:
extr=extractor.settings
label=extractor.settings.get('label')
label

1

In [67]:
al_dat_slice.to_csv('./liver_tumors_slices_LLB.csv')
al_dat_slice.to_excel('./multislice_excel_with_shape_2D.xlsx', index = False)

In [115]:
print(al_dat_slice)

  diagnostics_Versions_PyRadiomics diagnostics_Versions_Numpy  \
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                           v3.0.1                     1.24.3   
0                        